In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("langchain-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
langchain-Memory


In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [6]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)


abc123


AIMessage(content='Cosine represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 47, 'total_tokens': 66, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-691fcc85-106a-4822-9778-2c342b598f76-0', usage_metadata={'input_tokens': 47, 'output_tokens': 19, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)


abc123


AIMessage(content='코사인은 직각삼각형에서 인접변과 빗변의 비율을 나타냅니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 91, 'total_tokens': 131, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca6e63df-6812-4312-8343-4e61204ac942-0', usage_metadata={'input_tokens': 91, 'output_tokens': 40, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [8]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)

def234


AIMessage(content='수학에 능숙한 어시스턴트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-afd7dde4-077b-4db7-a8b2-f6ddf484650d-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [9]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[  # 기존의 "session_id" 설정을 대체하게 됩니다.
        ConfigurableFieldSpec(
            id="user_id",  # get_session_history 함수의 첫 번째 인자로 사용됩니다.
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # get_session_history 함수의 두 번째 인자로 사용됩니다.
            annotation=str,
            name="Conversation ID",
            description="대화의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
    ],
)


In [10]:
with_message_history.invoke(
    # ability와 input을 포함한 딕셔너리를 전달.
    {"ability": "math", "input": "What does cosine mean?"},
    # config 딕셔너리를 전달.
    config={"configurable": {"user_id": "abc123", "conversation_id": "1"}},
)

AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 47, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-851840be-82a3-4f32-8979-8c8a4ea48140-0', usage_metadata={'input_tokens': 47, 'output_tokens': 26, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [11]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOpenAI()})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]


# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)


{'output_message': AIMessage(content='In mathematics, the cosine of an angle in a right-angled triangle is the ratio of the length of the side adjacent to the angle to the length of the hypotenuse. It is one of the basic trigonometric functions and is often abbreviated as cos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 14, 'total_tokens': 67, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d179cf4-3ae7-4804-bb97-888b6410dee5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 53, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})}

In [12]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="이전의 내용을 한글로 답변해 주세요!")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='수학에서 직각삼각형의 각도의 코사인은 그 각도에 인접한 변의 길이와 빗변의 길이의 비율입니다. 이것은 기본 삼각함수 중 하나이며 종종 cos로 약식으로 표시됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 92, 'total_tokens': 182, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a309149e-36e4-429d-839b-8a57c091ad59-0', usage_metadata={'input_tokens': 92, 'output_tokens': 90, 'total_tokens': 182, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})}

In [13]:
with_message_history = RunnableWithMessageHistory(
    ChatOpenAI(),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)


In [14]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "def123"}},
)


AIMessage(content='코사인은 삼각함수 중 하나로, 직각 삼각형의 두 변의 길이를 이용하여 각도를 계산하는 함수입니다. 코사인은 직각 삼각형에서 빗변과 인접 변의 길이의 비를 나타내며, 각도가 주어졌을 때 코사인 값은 삼각형의 변들 사이의 각도 관계를 설명하는 중요한 값 중 하나입니다. 코사인은 주로 수학, 물리학, 공학 등 다양한 분야에서 사용되며, 삼각함수 중에서도 가장 널리 사용되는 함수 중 하나입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 24, 'total_tokens': 224, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-53cb6f24-7681-4dc8-8f3b-34989581ae88-0', usage_metadata={'input_tokens': 24, 'output_tokens': 200, 'total_tokens': 224, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [15]:
from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)


In [16]:
with_message_history.invoke(
    {"input_messages": "코사인의 의미는 무엇인가요?"},
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "xyz123"}},
)


AIMessage(content='코사인은 삼각함수 중 하나로, 직각 삼각형에서 두 변의 길이를 이용하여 각도를 구하는 방법 중 하나입니다. 삼각형의 밑변과 높이의 비율을 나타내는 것으로, 각도가 주어졌을 때 코사인은 삼각형의 밑변과 빗변의 비율을 나타냅니다. 코사인 함수는 주어진 각도에 대한 코사인 값을 계산하는 함수이며, 삼각형의 변의 길이를 알고 있는 경우에 각도를 구하는 데 사용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 24, 'total_tokens': 215, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22116f80-eb82-41e0-a1d8-97a30d3603b9-0', usage_metadata={'input_tokens': 24, 'output_tokens': 191, 'total_tokens': 215, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [17]:
# Redis 서버의 URL을 지정합니다.
REDIS_URL = "redis://localhost:6379/0"


In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

# LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# LANGCHAIN_PROJECT 설정
os.environ["LANGCHAIN_PROJECT"] = "RunnableWithMessageHistory"


In [19]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 세션 ID를 기반으로 RedisChatMessageHistory 객체를 반환합니다.
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,  # 실행 가능한 객체
    get_message_history,  # 메시지 기록을 가져오는 함수
    input_messages_key="input",  # 입력 메시지의 키
    history_messages_key="history",  # 기록 메시지의 키
)

In [20]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 옵션으로 세션 ID를 "redis123" 로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)


AIMessage(content='Cosine represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 47, 'total_tokens': 66, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-146afcbf-7301-4673-b3d4-0bf6ae2c9191-0', usage_metadata={'input_tokens': 47, 'output_tokens': 19, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [21]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "foobar"로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)

AIMessage(content='코사인은 직각 삼각형에서 인접변과 빗변의 비율을 나타냅니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 92, 'total_tokens': 133, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-986633f9-22a6-4c16-89d9-3918780b585d-0', usage_metadata={'input_tokens': 92, 'output_tokens': 41, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [22]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "redis456"로 지정합니다.
    config={"configurable": {"session_id": "redis456"}},
)

AIMessage(content='수학에 능숙한 도우미입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c7ce147f-2ac9-4068-8405-6bc91669a52e-0', usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})